In [1]:
import pandas as pd
import pulp

df = pd.read_excel("data.xlsx")
df['predecessorTaskIDs'].fillna("", inplace=True)
df = df[['taskID', 'task', 'predecessorTaskIDs', 'bestCaseHours', 'expectedHours', 'worstCaseHours']]

df

,taskID,task,predecessorTaskIDs,bestCaseHours,expectedHours,worstCaseHours
0,A,Describe product,,1.0,2.0,3.0
1,B,Develop marketing strategy,,1.0,2.0,4.0
2,C,Design brochure,A,2.0,4.0,6.0
3,D,Develop product prototype,,NaN,NaN,NaN
4,D1,Requirements analysis,A,2.0,4.0,8.0
5,D2,Software design,D1,10.0,16.0,24.0
6,D3,System design,D1,10.0,16.0,24.0
7,D4,Coding,"D2, D3",10.0,16.0,24.0
8,D5,Write documentation,D4,2.0,4.0,8.0
9,D6,Unit testing,D4,16.0,24.0,32.0


In [14]:


def solve_project_plan(df, case="expected"):
    # LP Model
    lp_problem = pulp.LpProblem("Project_Planning", pulp.LpMinimize)
    
    # Decision vars
    start_times = pulp.LpVariable.dicts("start_time", df["taskID"], 0, None)
    
    # Objective function
    project_end = pulp.LpVariable("project_end", 0, None)
    lp_problem += project_end, "Minimize_Project_Duration"
    
    # adding constraints
    for index, row in df.iterrows():
        task = row["taskID"]
        
        # duration of cases
        if case == "best":
            duration = row["bestCaseHours"]
        elif case == "worst":
            duration = row["worstCaseHours"]
        else:
            duration = row["expectedHours"]
        
        predecessors = [pred.strip() for pred in row["predecessorTaskIDs"].split(",") if pred.strip()]
        
        # task duration constraint
        lp_problem += start_times[task] + duration <= project_end
        
        # task dependencies constraints
        for predecessor in predecessors:
            lp_problem += start_times[task] >= start_times[predecessor] + duration

    # Solver
    lp_problem.solve(pulp.GLPK_CMD())
    
    # output prep
    results = {
        "status": pulp.LpStatus[lp_problem.status],
        "optimal_duration": project_end.varValue,
        "task_start_times": {task: var.varValue for task, var in start_times.items()}
    }
    
    return results

In [15]:
cases = ["best", "expected", "worst"]
for case in cases:
    result = solve_project_plan(df, case)
    print(f"Results for {case} case:")
    print(f"Optimal project duration: {result['optimal_duration']} hours")
    print("Start times for tasks:")
    for task, start_time in result["task_start_times"].items():
        print(f"Task {task}: {start_time} hours")
    print("\n")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var/folders/s2/q9mbkp214x35m1r7fqtd0d840000gn/T/49e298a671d947a69d2f4dfd16d86613-pulp.lp
 -o /var/folders/s2/q9mbkp214x35m1r7fqtd0d840000gn/T/49e298a671d947a69d2f4dfd16d86613-pulp.sol
Reading problem data from '/var/folders/s2/q9mbkp214x35m1r7fqtd0d840000gn/T/49e298a671d947a69d2f4dfd16d86613-pulp.lp'...
/var/folders/s2/q9mbkp214x35m1r7fqtd0d840000gn/T/49e298a671d947a69d2f4dfd16d86613-pulp.lp:35: missing right-hand side
CPLEX LP file processing error


PulpSolverError: PuLP: Error while executing glpsol